In [1]:
import os
import openai
import pandas as pd
import PyPDF2 as pdf
import re
from dotenv import load_dotenv
from tqdm import tqdm
import json

In [41]:
load_dotenv()
DATA_FOLDER = 'dane'
OPENAI_KEY  = os.getenv('OPENAI_KEY')

openai.api_key = OPENAI_KEY

PATTERN = r'Zad\.\s[0-9]\.(.*?)\n'

In [ ]:
def read_tasks(text, pattern):
    tasks = []
    matches = re.findall(pattern, text, flags=re.DOTALL)
    for match in matches:
        tasks.append(match.strip())

    return tasks

In [43]:
def read_pdf(path):
    pdfFileObj = open(path, 'rb')
    pdf_reader = pdf.PdfReader(pdfFileObj)
    pageObj = pdf_reader.pages[0]
    return pageObj.extract_text()

In [59]:
def prompt_gpt(question):
    messages = [{"role": "user", "content": question}]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    return response.choices[0].message.content

In [60]:
def main():
    solutions = {}
    for f in tqdm(os.listdir(DATA_FOLDER)):
        if f.endswith('.pdf'):
            file_number = f.split('.')[0]
            key = f"## Plik. {file_number}"
            solutions[key] = {}     
            text = read_pdf(os.path.join(DATA_FOLDER, f))
            tasks = read_tasks(text, PATTERN)

            solutions[key]['q&a'] = []
            solutions[key]['#n'] = int(file_number)

            for n, task in enumerate(tasks):
                try:
                    answer = prompt_gpt(task)
                except:
                    answer = 'N/A'
                    
                solutions[key]['q&a'].append({f'Zad. {n+1} {task}': answer})
    # sort dict by values
    solutions = dict(sorted(solutions.items(), key=lambda x: x[1]['#n'])) 

    with open('kolo.json', 'w') as f:
        json.dump(solutions, f)


main()

100%|██████████| 45/45 [1:04:05<00:00, 85.45s/it]


### **uporządkowanie struktury pliku json -> txt**

In [4]:
content = json.load(open('kolo.json', 'r'))
one_big_string = ''
for k, v in content.items():
    one_big_string += k + '\n'
    for p in v['q&a']:
        for k, v in p.items():
            one_big_string += '**' + k + '**' + '\n'
            one_big_string += '\n' + v + '\n'
        one_big_string += '\n'

with open('kolo.md', 'w') as f:
    f.write(one_big_string)